In [ ]:
# Current State - 
# 5 days 
# MAX_DATE = '2020-01-01'
# MIN_DATE = '2018-11-01'
# With normalization

# RF 0.6634615384615384
# LR 0.5769230769230769



# ----------------------

In [1]:
import numpy as np
def get_possessions_per_game_and_team(row,pbp):
    try:
        GAME_ID = row['GAME_ID']
        is_home = row['IsHome']
        single_game = pbp[pbp['GAME_ID']==GAME_ID]
        index_indicator = 'Home_EOP' if is_home==1 else 'Visitor_EOP'

        single_game = single_game.sort_values('EVENTNUM')
        single_game['Shift_HOMEDESCRIPTION'] = single_game['HOMEDESCRIPTION'].shift(-1)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION']]
        single_game.loc[(single_game['HOMEDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_HOMEDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Home_EOP'] = 1
        single_game['Home_EOP'].fillna(0,inplace=True)
        single_game[['Shift_HOMEDESCRIPTION','HOMEDESCRIPTION','VISITORDESCRIPTION','Home_EOP']]

        single_game['Shift_VISITORDESCRIPTION'] = single_game['VISITORDESCRIPTION'].shift(-1)
        single_game[['Shift_VISITORDESCRIPTION','VISITORDESCRIPTION']]
        single_game.loc[(single_game['VISITORDESCRIPTION'].isnull()==False)&
                        (single_game['Shift_VISITORDESCRIPTION'].isnull()==True)&
                        (single_game['EVENTMSGTYPE'].isin([1,2,3,4,5,7,9])),'Visitor_EOP'] = 1
        single_game['Visitor_EOP'].fillna(0,inplace=True)
        return single_game[['Home_EOP','Visitor_EOP']].sum().loc[index_indicator]
    except:
        return np.nan




In [11]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import itertools


nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
       'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
       'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
       'WAS', 'ORL', 'GSW', 'NOP', 
       'MIA', 
       'DET']
nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
       1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
       1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
       1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
       1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
       1610612753, 1610612747, 1610612758, 1610612748, 1610612765]





nba_teams_df = pd.DataFrame(nba_teams)
dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')

def load_dfs( MAX_DATE = None , MIN_DATE = None):
    nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
           'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
           'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
           'WAS', 'ORL', 'GSW', 'NOP', 
           'MIA', 
           'DET']
    nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
           1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
           1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
           1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
           1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
           1610612753, 1610612747, 1610612758, 1610612748, 1610612765]

   



    nba_teams_df = pd.DataFrame(nba_teams)
    dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')


    df = pd.read_csv('large_boxscoretraditionalv2_df.csv')
    df=df[df['TEAM_ID'].isin(nba_team_ids)]
    df.drop_duplicates(subset = ['GAME_ID','PLAYER_ID'],inplace=True)
    df.dropna(subset=['MIN'],inplace=True)



    df_g = pd.read_csv('all_games.csv')
    df_g.drop_duplicates(subset =['TEAM_ID','GAME_ID'],inplace=True)

    df_g.loc[df_g['WL']=='W','IsWin']=1
    df_g.loc[df_g['WL']=='L','IsWin']=0
    df_g['IsHome'] = (df_g['MATCHUP'].str.contains('vs.')).astype(int)
    df_g['Opposing_team'] = df_g['MATCHUP'].apply( lambda x: x.split(' ')[2])

    df_g=df_g[df_g['TEAM_ID'].isin(nba_team_ids)]
    team_abv_to_id_mapping = df_g[['TEAM_ABBREVIATION','TEAM_ID']].drop_duplicates().set_index('TEAM_ABBREVIATION')['TEAM_ID'].to_dict()
    df_g['Opposing_team_ID'] = df_g['Opposing_team'].apply(lambda x: team_abv_to_id_mapping[x] if x in team_abv_to_id_mapping else None )

    df_g=df_g[df_g['Opposing_team_ID'].isin(nba_team_ids)]

    # df_g=df_g[['GAME_ID','TEAM_ID','GAME_DATE']].drop_duplicates().dropna()
    df_g['GAME_DATE'] = pd.to_datetime(df_g['GAME_DATE'])
    
    if MAX_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']<MAX_DATE]
    if MIN_DATE is not None:
        df_g = df_g[df_g['GAME_DATE']>MIN_DATE]

    # add balance scoring


    df = df.merge(df_g, 
         how='inner',
         left_on=['GAME_ID','TEAM_ID'],
         right_on=['GAME_ID','TEAM_ID'])


    pbp = pd.read_csv('large_playbyplayv2_df.csv', index_col=0)
    pbp.drop_duplicates(subset=['GAME_ID','EVENTNUM','PERIOD'], inplace=True)


    shot_chart_df =  pd.read_csv('ShotChartDetail_v2.csv')
    shot_chart_df.drop_duplicates(subset=['GAME_ID','GAME_EVENT_ID'], inplace=True)
    
    
    df_g['num_of_possessions'] = df_g.apply( lambda x: get_possessions_per_game_and_team(x,pbp), axis=1)
    df_g.dropna(subset=['num_of_possessions'],inplace=True)
    df_g['AdjustedPM'] = (df_g['PLUS_MINUS']/df_g['num_of_possessions'])*100
    df_g['OffRating'] = (df_g['PTS']/df_g['num_of_possessions'])*100
    df_g['EFG'] = (df_g['FGM'] + 0.5*df_g['FG3M'])/df_g['FGA']
    df_g['AST_ratio'] = df_g['AST']*100/((df_g['FGA'])+(df_g['FTA']*0.44)+(df_g['AST'])+(df_g['AST']))
    df_g['Opp_points'] = df_g['PTS'] + df_g['PLUS_MINUS']
    df_g['Def_Rating'] = (df_g['Opp_points'] / df_g['num_of_possessions'])*100

    ['AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']



    print('done')
    return df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping

df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs(  MAX_DATE = '2020-02-01',MIN_DATE = '2019-01-01')

done


In [12]:
nba_teams = ['MIL', 'CHI', 'CHA', 'TOR', 'BOS', 'PHX', 'OKC', 'LAC', 'IND',
       'BKN', 'MIN', 'UTA', 'SAS', 'DAL', 'CLE', 'NYK', 'POR', 'HOU',
       'DEN', 'MEM', 'SAC', 'PHI',  'ATL', 'LAL', 
       'WAS', 'ORL', 'GSW', 'NOP', 
       'MIA', 
       'DET']
nba_team_ids =[1610612749, 1610612766, 1610612738, 1610612746, 1610612754,
       1610612750, 1610612741, 1610612742, 1610612762, 1610612759,
       1610612739, 1610612752, 1610612761, 1610612760, 1610612757,
       1610612751, 1610612745, 1610612756, 1610612743, 1610612755,
       1610612737, 1610612763, 1610612764, 1610612744, 1610612740,
       1610612753, 1610612747, 1610612758, 1610612748, 1610612765]





nba_teams_df = pd.DataFrame(nba_teams)
dummy_nba_teams= pd.get_dummies(nba_team_ids, prefix ='team')

sliding_window_num_of_games = 5

large_df_of_sequences = pd.DataFrame([])
 
# create the sliding window dataframe per team
for team in nba_team_ids:
    print(team)
    x = df_g[df_g['TEAM_ID'] == team].sort_values('GAME_DATE').reset_index(drop=True)
    
    
    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values
    
    
    list_team_indicator = list(team_indicator) 
    for ix,row in x[sliding_window_num_of_games:].iterrows():
        tmp = x.loc[ix-sliding_window_num_of_games:ix]
        
        # add sequence of wins
        list_of_sequence_of_wins = list(tmp['IsWin'].values[:-1])
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
        df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        # get team indicator
        team_indicator = dummy_curr_team[0:1]
        
        # is at home
        list_of_sequence_of_at_home = list(tmp['IsHome'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games+1):
                new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
        df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
        
        # get opp team indicator 
        opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
        opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
        appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
        appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])
        
        
        # add averages of categories
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating'
                       ]
        averages_of_categories_curr_team = tmp[cols_to_calc][:-1].mean().values
        df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns =cols_to_calc)
        
        
        # add shot area avgs for curr team
        shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
       'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']==team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add shot chart of opposing teams during window
        shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                                 (shot_chart_df['TEAM_ID']!=team)].groupby(
                                    ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
        shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
        shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
        
        # add sequence of scores
        list_of_sequence_of_wins = list(tmp['PTS'].values)
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_Scores'.format(game + 1))
        new_cols.append('Y') 
        df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
        
        # add sequence of scores of opp
        list_of_sequence_of_opp_scores = list(tmp['Opp_points'].values[:-1])
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_Scores_Opp'.format(game + 1))
        df_of_sequence_of_scores_opp = pd.DataFrame([list_of_sequence_of_opp_scores],columns=new_cols)
        
        
         # add averages of categories for past opponents
        list_of_opp_ids = list(tmp['Opposing_team_ID'].unique())

        x_opp = df_g[(df_g['TEAM_ID'].isin(list_of_opp_ids))&
                    (df_g['GAME_DATE'] < tmp['GAME_DATE'].max())&
                    (df_g['GAME_DATE'] >= tmp['GAME_DATE'].min())].sort_values('GAME_DATE').reset_index(drop=True)
        tmp_opp = x_opp[-sliding_window_num_of_games:ix]
        cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
        averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
        df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns = cols_to_calc).reset_index(drop=True)
        
        # add sequence of days between games
        tmp['GAME_DATE'].diff().dt.days.dropna()
        l = tmp['GAME_DATE'].diff().dt.days.dropna().values
        l = [4 if sl >4 else sl for sl in l]
        new_cols = []
        for game in range(sliding_window_num_of_games):
                new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))

        df_days_between = pd.DataFrame([l],columns=new_cols)

        # start merging all together 
        # 1. wins
        # 2. team indicator
        # 3. at home indicator
        # 4. TOV sequence
#         print(len(team_indicator.columns),'team_indicator')
#         print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
#         print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
#         print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
#         print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
#         print(len(shot_area_avgs.columns),'shot_area_avgs')
#         print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
        
        df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)
        df_all_features = df_all_features.merge(df_of_sequence_of_scores_opp, how='inner' , left_index=True , right_index=True)

        large_df_of_sequences = large_df_of_sequences.append(df_all_features)
    
large_df_of_sequences.dropna(inplace=True)



1610612749
1610612766
1610612738
1610612746
1610612754
1610612750
1610612741
1610612742
1610612762
1610612759
1610612739
1610612752
1610612761
1610612760
1610612757
1610612751
1610612745
1610612756
1610612743
1610612755
1610612737
1610612763
1610612764
1610612744
1610612740
1610612753
1610612747
1610612758
1610612748
1610612765


In [4]:
large_df_of_sequences.shape

(1711, 283)

In [4]:
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn import preprocessing

#scale the X variables
X = large_df_of_sequences.loc[:, large_df_of_sequences.columns != 'Y']
saved_columns = X.columns
x = X.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
X = pd.DataFrame(x_scaled, columns=saved_columns)


Y = large_df_of_sequences['Y']

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=10)

rf = RandomForestRegressor(n_estimators=1000, random_state=1)
rf.fit(x_train,y_train)


# Use the forest's predict method on the test data
predictions = rf.predict(x_test)
# Calculate the absolute errors
errors = abs(predictions - y_test)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')


# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / y_test)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
    
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 4)) for feature, importance in zip(X.columns, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];    

y_predict = rf.predict(x_test)
# print ("R2 score:",r2_score(y_test,y_predict))


Mean Absolute Error: 9.78 degrees.
Accuracy: 90.88 %.
Variable: PTS_x                Importance: 0.0301
Variable: STL_y                Importance: 0.0231
Variable: num_of_possessions_y Importance: 0.0226
Variable: 2_games_back_Scores  Importance: 0.021
Variable: REB_x                Importance: 0.0207
Variable: In The Paint (Non-RA)_y Importance: 0.0191
Variable: num_of_possessions_x Importance: 0.0188
Variable: FT_PCT_x             Importance: 0.0182
Variable: Restricted Area_x    Importance: 0.0174
Variable: BLK_y                Importance: 0.0172
Variable: FG3_PCT_y            Importance: 0.0171
Variable: FG3_PCT_x            Importance: 0.0168
Variable: Above the Break 3_x  Importance: 0.0163
Variable: Mid-Range_x          Importance: 0.0159
Variable: In The Paint (Non-RA)_x Importance: 0.0159
Variable: 1_games_back_Scores_Opp Importance: 0.0159
Variable: 2_games_back_Scores_Opp Importance: 0.0157
Variable: FT_PCT_y             Importance: 0.0156
Variable: Mid-Range_y          Impo

In [13]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create param grid.

param_grid = {
    'bootstrap': [False],
    'max_depth': [110,150],
    'max_features': [20,200],
    'min_samples_leaf': [3],
    'min_samples_split': [ 5],
    'n_estimators': [  400]
}
rf = RandomForestRegressor()

# Create grid search object

clf = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

# Fit on data

best_clf = clf.fit(x_train, y_train)
from sklearn.metrics import classification_report
print("Best parameters set found on development set:")
print(best_clf.best_params_)
print("Detailed classification report:")
y_true, y_pred = y_test, best_clf.predict(x_test)


Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 out of  20 | elapsed:   20.2s remaining:   30.4s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   58.3s finished


Best parameters set found on development set:
{'bootstrap': False, 'max_depth': 110, 'max_features': 20, 'min_samples_leaf': 3, 'min_samples_split': 5, 'n_estimators': 400}
Detailed classification report:


In [7]:
from sklearn.metrics import r2_score

def RFR(X_train, X_test, y_train, y_test, best_params):
    from sklearn.ensemble import RandomForestRegressor
    estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
    print ("R2 score:",r2_score(y_test,y_predict))
    return y_test,y_predict


# best_score, best_params = Grid_Search_CV_RFR(X_train, y_train)
y_test , y_predict = RFR(x_train, x_test, y_train, y_test ,best_clf.best_params_ )
print ("Best Score:" ,best_clf.best_score_)


R2 score: 0.037294970747197564
Best Score: 0.027550110979266896


In [14]:
import statsmodels.api as sm
X_cons = sm.add_constant(X)
reg_model = sm.OLS(Y, X_cons.values).fit()
predictions = reg_model.predict(X_cons) 

print_model = reg_model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.327
Model:                            OLS   Adj. R-squared:                  0.202
Method:                 Least Squares   F-statistic:                     2.605
Date:                Thu, 13 Feb 2020   Prob (F-statistic):           6.12e-30
Time:                        16:12:07   Log-Likelihood:                -6411.8
No. Observations:                1711   AIC:                         1.336e+04
Df Residuals:                    1441   BIC:                         1.483e+04
Df Model:                         269                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         87.1491     10.159      8.578      0.0

In [7]:

def predict_game_score(
                        curr_team_abv,
                        opp_team,
                        is_current_game_at_home,
                        curr_team_id,
                        df_g,
                        df,
                        pbp,
                        shot_chart_df):
    
    
    
    
    x = df_g[df_g['TEAM_ID'] == curr_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp = x[-sliding_window_num_of_games:]

    dummy_curr_team = pd.get_dummies(x['TEAM_ID'], prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    dummy_curr_team.columns = ['curr_' + col for col in dummy_curr_team.columns]
    team_indicator = dummy_curr_team.loc[0].values

    list_of_sequence_of_wins = list(tmp['IsWin'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsWin'.format(game + 1))
    df_of_sequence_of_wins = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)

    # # get team indicator
    team_indicator = dummy_curr_team[0:1]
    team_indicator

    # is at home
    list_of_sequence_of_at_home = list(tmp['IsHome'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_IsHome'.format(game + 1))
    df_of_sequence_of_at_home = pd.DataFrame([list_of_sequence_of_at_home],columns=new_cols)
    df_of_sequence_of_at_home['{}_games_back_IsHome'.format(sliding_window_num_of_games+1)] = is_current_game_at_home


    # get opp team indicator 

    opp_dummy_teams = pd.get_dummies(tmp['Opposing_team_ID'].astype(int).append(pd.Series(opp_team_id)), prefix ='team').reindex(columns = dummy_nba_teams.columns, fill_value=0)
    opp_dummy_teams.columns = ['opp_' + col for col in opp_dummy_teams.columns]
    appended_list_of_encoded_opp_teams = list(itertools.chain(*opp_dummy_teams.values))
    appended_df_of_encoded_opp_teams = pd.DataFrame([appended_list_of_encoded_opp_teams])

    # add averages of categories
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_curr_team = tmp[cols_to_calc].mean().values
    df_of_averages_of_categories_curr_team = pd.DataFrame([averages_of_categories_curr_team],columns =cols_to_calc)
    df_of_averages_of_categories_curr_team

    # add shot area avgs for curr team
    shot_index= ['Mid-Range', 'In The Paint (Non-RA)', 'Restricted Area',
    'Above the Break 3', 'Left Corner 3', 'Right Corner 3']

    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']==curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs

    # add shot chart of opposing teams during window
    shot_chart_df_for_tmp = shot_chart_df[(shot_chart_df['GAME_ID'].isin(tmp[:-1]['GAME_ID'].values))&
                             (shot_chart_df['TEAM_ID']!=curr_team_id)].groupby(
                                ['SHOT_ZONE_BASIC']).mean()['SHOT_MADE_FLAG'].reset_index()
    shot_chart_df_for_tmp_reindex = shot_chart_df_for_tmp[['SHOT_ZONE_BASIC','SHOT_MADE_FLAG']].set_index('SHOT_ZONE_BASIC').reindex(shot_index).fillna(0)
    shot_area_avgs_opp = shot_chart_df_for_tmp_reindex.T.reset_index(drop=True)
    shot_area_avgs_opp
    
    
    # add sequence of scores
    list_of_sequence_of_wins = list(tmp['PTS'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_Scores'.format(game + 1))
    df_of_sequence_of_scores = pd.DataFrame([list_of_sequence_of_wins],columns=new_cols)
    
    # add sequence of scores of opp
    list_of_sequence_of_opp_scores = list(tmp['Opp_points'].values)
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_Scores_Opp'.format(game + 1))
    df_of_sequence_of_scores_opp = pd.DataFrame([list_of_sequence_of_opp_scores],columns=new_cols)
        
        
     # add averages of categories
    x_opp = df_g[df_g['TEAM_ID'] == opp_team_id].sort_values('GAME_DATE').reset_index(drop=True)
    tmp_opp = x_opp[-sliding_window_num_of_games:]
    cols_to_calc = ['PTS','REB','AST','STL','BLK','TOV','PLUS_MINUS','FG3_PCT','FG_PCT','FT_PCT','num_of_possessions',
                       'AdjustedPM','OffRating','EFG','AST_ratio','Opp_points','Def_Rating']
    averages_of_categories_opp_team = tmp_opp[cols_to_calc].mean().values
    df_of_averages_of_categories_opp_team = pd.DataFrame([averages_of_categories_opp_team],columns =cols_to_calc)
    df_of_averages_of_categories_opp_team
    
    # add sequence of days between games
    tmp_days_back = x[-sliding_window_num_of_games-1:]
    l = tmp_days_back['GAME_DATE'].diff().dt.days.dropna().values
    l = [4 if sl >4 else sl for sl in l]
    new_cols = []
    for game in range(sliding_window_num_of_games):
            new_cols.insert(0,'{}_games_back_PTS'.format(game + 1))
    df_days_between = pd.DataFrame([l],columns=new_cols)

    # start merging all together 
    # 1. wins
    # 2. team indicator
    # 3. at home indicator
    # 4. TOV sequence
    # print(len(team_indicator.columns),'team_indicator')
    # print(len(df_of_sequence_of_wins.columns),'df_of_sequence_of_wins')
    # print(len(df_of_sequence_of_at_home.columns),'df_of_sequence_of_at_home')
    # print(len(appended_df_of_encoded_opp_teams.columns),'appended_df_of_encoded_opp_teams')
    # print(len(df_of_averages_of_categories_curr_team.columns),'df_of_averages_of_categories_curr_team')
    # print(len(shot_area_avgs.columns),'shot_area_avgs')
    # print(len(shot_area_avgs_opp.columns),'shot_area_avgs_opp')
    df_all_features = team_indicator.merge(df_of_sequence_of_wins , how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_at_home, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(appended_df_of_encoded_opp_teams, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_curr_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(shot_area_avgs_opp, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_scores, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_averages_of_categories_opp_team, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_days_between, how='inner' , left_index=True , right_index=True)
    df_all_features = df_all_features.merge(df_of_sequence_of_scores_opp, how='inner' , left_index=True , right_index=True)

    len(df_all_features.columns)
    
#     len(X.columns)

#     print([curr_team_abv,'------->' , best_clf.predict(df_all_features) , '-------',
#                         opp_team,
#                         is_current_game_at_home,
#                         curr_team_id])
#     print()
#     df_all_features = df_all_features.loc[:, df_all_features.columns != 'Y']
    df_all_features = df_all_features[X.columns]

    df_all_features = pd.DataFrame(min_max_scaler.transform(df_all_features),columns=df_all_features.columns)
    return df_all_features, best_clf.predict(df_all_features)

In [15]:
import pickle
filename_rf = 'RF_basic_2019-01-01_2020-02-01_5Days.sav'
pickle.dump(best_clf, open(filename_rf,'wb'))
best_clf = pickle.load(open(filename_rf, 'rb'))


filename_lr = 'LR_basic_2019-01-01_2020-02-01_5Days.sav'
pickle.dump(reg_model, open(filename_lr,'wb'))
reg_model = pickle.load(open(filename_lr, 'rb'))


In [16]:
# lsit_of_games = [
#                 ['TOR','POR'] , 
#                 ['CLE','DET'] , 
#                 ['BKN','OKC'] , 
#                 ['MEM','MIN'] , 
#                 ['PHX','SAC'] , 
#                 ['LAL','NYK'] , 
                
#             ]

# df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = '2020-01-07')

total_results_saved = []


FIRST_DATE_TO_CHECK = '2020-02-01'
LAST_DATE_TO_RUN_ON = '2020-02-13'

df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)

list_of_games_to_check = df_g_global[['GAME_DATE','TEAM_ABBREVIATION','GAME_ID','IsHome']]
len(list_of_games_to_check)
merged_list_of_games_to_check = list_of_games_to_check.merge(list_of_games_to_check,
                                                             how='inner',
                                                             left_on =['GAME_ID'] ,
                                                             right_on =['GAME_ID'])
merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['TEAM_ABBREVIATION_x']!= merged_list_of_games_to_check['TEAM_ABBREVIATION_y']]
merged_list_of_games_to_check=merged_list_of_games_to_check[merged_list_of_games_to_check['IsHome_x']==1]


merged_list_of_games_to_check = merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']>FIRST_DATE_TO_CHECK]
list_of_dates = merged_list_of_games_to_check['GAME_DATE_x'].unique()
for single_date in list_of_dates:
    print (single_date)
    lsit_of_games = []
    for ix,row in merged_list_of_games_to_check[merged_list_of_games_to_check['GAME_DATE_x']==single_date].iterrows():
        game=[row['TEAM_ABBREVIATION_x'],row['TEAM_ABBREVIATION_y']]
        lsit_of_games.append(game)
    df,df_g,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = str(single_date)[:10])
     

    sliding_window_num_of_games = 5

    for game in lsit_of_games:
    #     print(df_g['GAME_DATE'].max())
    #     df_g['TEAM_ABBREVIATION'].unique()
        print(game)
        print(df_g['GAME_DATE'].max())
        df_g['TEAM_ABBREVIATION'].unique()
        ###########################
        ### DEFINE YOUR TEAMS   ###
        ###########################
        curr_team_abv = game[0] #'PHX'
        curr_team_id = team_abv_to_id_mapping[curr_team_abv]

        opp_team = game[1] #'SAC'
        opp_team_id  = team_abv_to_id_mapping[opp_team]
        is_current_game_at_home = 1



        ###########################


        res_1,rf_score_1 = predict_game_score(
                            curr_team_abv,
                           opp_team,
                           1,
                           curr_team_id ,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
        res_1['const'] = 1
        reg_score_1 = reg_model.predict(res_1[X_cons.columns])
    #     print('RES' , reg_model.predict(res_1[X_cons.columns]))
        res_2 , rf_score_2 = predict_game_score(
                            opp_team,
                           curr_team_abv,
                           0,
                           opp_team_id,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
        res_2['const'] = 1
        reg_score_2 =  reg_model.predict(res_2[X_cons.columns])

        print('RF:' , rf_score_1[0],':',rf_score_2[0] )
        print ('LR:' , reg_score_1.values[0],':',reg_score_2.values[0] )
        print()
        results_saved = [single_date,curr_team_abv,opp_team,
                        rf_score_1[0],rf_score_2[0],
                         reg_score_1.values[0],reg_score_2.values[0]]
        total_results_saved.append(results_saved)

done
2020-02-02T00:00:00.000000000
done
['TOR', 'CHI']
2020-02-01 00:00:00
RF: 110.99050000000005 : 109.13955952380948
LR: 113.89736066641848 : 101.14887484511223

['HOU', 'NOP']
2020-02-01 00:00:00
RF: 112.60188888888884 : 114.53783333333331
LR: 119.91544284492622 : 117.83664679850492

['DET', 'DEN']
2020-02-01 00:00:00
RF: 109.32218452380948 : 109.98028739316234
LR: 99.64991811261973 : 93.75123955504554

['MIL', 'PHX']
2020-02-01 00:00:00
RF: 113.9619404761905 : 111.44788446969706
LR: 127.10790650347121 : 117.19276208734806

2020-02-03T00:00:00.000000000
done
['ATL', 'BOS']
2020-02-02 00:00:00
RF: 112.8283640350877 : 108.5289230769232
LR: 113.45411503334633 : 101.29463434180064

['CHA', 'ORL']
2020-02-02 00:00:00
RF: 105.12852564102563 : 107.49086309523817
LR: 97.01267848554777 : 99.30806825907302

['SAC', 'MIN']
2020-02-02 00:00:00
RF: 112.84247222222231 : 113.995875
LR: 124.01938785893138 : 123.7385905619594

['MIA', 'PHI']
2020-02-02 00:00:00
RF: 110.67922916666663 : 112.646484217

RF: 112.25302380952388 : 112.84837499999998
LR: 126.10930087218523 : 124.05955860901915

2020-02-12T00:00:00.000000000
done
['DEN', 'LAL']
2020-02-11 00:00:00
RF: 113.03885416666671 : 112.4322692307692
LR: 99.68117113860123 : 114.91069010711136

['ORL', 'DET']
2020-02-11 00:00:00
RF: 105.26682638888889 : 106.05063095238101
LR: 101.84859148043867 : 105.752898922443

['CLE', 'ATL']
2020-02-11 00:00:00
RF: 109.98919047619043 : 112.97631688596495
LR: 114.02937236868986 : 116.0611226790416

['UTA', 'MIA']
2020-02-11 00:00:00
RF: 108.20959866814472 : 109.43595833333333
LR: 105.26883023143179 : 98.57752435077663

['NYK', 'WAS']
2020-02-11 00:00:00
RF: 113.26439556277064 : 117.01329166666672
LR: 108.25979746136765 : 134.3424520385028

['MIN', 'CHA']
2020-02-11 00:00:00
RF: 111.82045833333335 : 107.33790476190475
LR: 109.73556583041619 : 102.33219552799679

['BKN', 'TOR']
2020-02-11 00:00:00
RF: 110.24439583333329 : 108.1843958333333
LR: 104.22773563273824 : 110.61787369239863

['MEM', 'POR']
2

In [17]:
pd.DataFrame(total_results_saved
            ,columns=['Date','HomeTeam','AwayTeam','RFHome','RFAway','LRHome','LRAway']).to_csv('~/basic_RF_LR_Jan.csv',index=False, mode='a', header=False)
# .to_csv('~/probs_Jan.csv',index=False)


In [21]:
################
# RUN PREDICTED
################

LAST_DATE_TO_RUN_ON = '2020-02-13' # not including

df,df_g_global ,pbp,shot_chart_df,team_abv_to_id_mapping = load_dfs( MAX_DATE = LAST_DATE_TO_RUN_ON)
           
lsit_of_games = [
['BOS','LAC'],
['NOP','OKC'],
]


        

sliding_window_num_of_games = 5

for game in lsit_of_games:
    print(game)
#     print(df_g['GAME_DATE'].max())
#     df_g['TEAM_ABBREVIATION'].unique()
    print(game)
    print(df_g_global['GAME_DATE'].max())
    df_g['TEAM_ABBREVIATION'].unique()
    ###########################
    ### DEFINE YOUR TEAMS   ###
    ###########################
    curr_team_abv = game[0] #'PHX'
    curr_team_id = team_abv_to_id_mapping[curr_team_abv]

    opp_team = game[1] #'SAC'
    opp_team_id  = team_abv_to_id_mapping[opp_team]
    is_current_game_at_home = 1



    ###########################


    res_1,rf_score_1 = predict_game_score(
                            curr_team_abv,
                           opp_team,
                           1,
                           curr_team_id ,
                          df_g,
                          df,
                          pbp,
                          shot_chart_df)
    #     res = sm.add_constant(res)
    res_1['const'] = 1
    reg_score_1 = reg_model.predict(res_1[X_cons.columns])
#     print('RES' , reg_model.predict(res_1[X_cons.columns]))
    res_2 , rf_score_2 = predict_game_score(
                        opp_team,
                       curr_team_abv,
                       0,
                       opp_team_id,
                      df_g,
                      df,
                      pbp,
                      shot_chart_df)
#     res = sm.add_constant(res)
    res_2['const'] = 1
    reg_score_2 =  reg_model.predict(res_2[X_cons.columns])

    print('RF:' , rf_score_1[0],':',rf_score_2[0] )
    print ('LR:' , reg_score_1.values[0],':',reg_score_2.values[0] )
    print()

done
['BOS', 'LAC']
['BOS', 'LAC']
2020-02-12 00:00:00
RF: 109.59694940476193 : 111.61624145299149
LR: 109.82216469458058 : 118.17712560853886

['NOP', 'OKC']
['NOP', 'OKC']
2020-02-12 00:00:00
RF: 112.63562878787872 : 108.72373862044824
LR: 118.83385394243604 : 107.37488194416616

